In [1]:
# import libraries
import re
import zipfile
import getpass
from osgeo import gdal 
import os  # for chdir, getcwd, path.basename, path.exists
import pandas as pd # for DatetimeIndex
import codecs # for text parsing code
import netrc
import rasterio as rio
import glob
import io

In [2]:
# Get NASA EARTHDATA Credentials from ~/.netrc or manual input
try:
    os.chmod('/home/jovyan/.netrc', 0o600) #only necessary on jupyterhub
    (ASF_USER, account, ASF_PASS) = netrc.netrc().authenticators("urs.earthdata.nasa.gov")
except:
    ASF_USER = input("Enter Username: ")
    ASF_PASS = getpass.getpass("Enter Password: ")

In [3]:
if not os.path.exists('/tmp/'):
    os.chdir('/tmp')
   
# directory for data downloads

data_dir = os.path.join('/tmp')
os.makedirs(data_dir, exist_ok=True)
print(data_dir)

/tmp


In [ ]:
%%time 

files = ['https://datapool.asf.alaska.edu/PROJECTED/UA/lowman_05208_21015_009_210303_L090_CX_01_grd.zip']
    
for file in files:
    print(f'downloading {file}...')
    filename = os.path.basename(file)
    
    if not os.path.exists(os.path.join(data_dir,filename)):
        cmd = "wget -q {0} --user={1} --password={2} -P {3} -nc".format(file, ASF_USER, ASF_PASS, data_dir)
        os.system(cmd) ##Should probably be a subprocess.call(cmd) - not quite sure why but that is the perfered method
    else:
        print(filename + " already exists. Skipping download ..")
        
print("done")

In [4]:
# check to see if downloaded
# the *.* syntax means print all files in the directory

print(glob.glob("/tmp/*.*"))

['/tmp/lowman_05208_21015_009_210303_L090_CX_01_grd.zip']


In [5]:
# unzip

for file in glob.glob("/tmp/*.*"):
    with zipfile.ZipFile(file, "r") as zip_ref:
        zip_ref.printdir()
        print('Extracting all the files now...')
        zip_ref.extractall('/tmp')
        print("done")
    


File Name                                             Modified             Size
lowman_05208_21015_009_210303_L090HHHH_CX_01.grd 2021-03-16 18:02:16   1738168256
lowman_05208_21015_009_210303_L090HHHV_CX_01.grd 2021-03-16 18:12:06   3476336512
lowman_05208_21015_009_210303_L090HHVV_CX_01.grd 2021-03-16 18:13:40   3476336512
lowman_05208_21015_009_210303_L090HVHV_CX_01.grd 2021-03-16 18:15:14   1738168256
lowman_05208_21015_009_210303_L090HVVV_CX_01.grd 2021-03-16 18:16:00   3476336512
lowman_05208_21015_009_210303_L090VVVV_CX_01.grd 2021-03-16 18:17:34   1738168256
lowman_05208_21015_009_210303_L090_CX_01.ann   2021-03-16 18:18:22        29028
Extracting all the files now...
done


In [6]:
# clean up unwanted data from what we just downloaded

directory = '/tmp'
os.chdir(directory)
HV_files = glob.glob('*HV_01*') #define all HV
VV_files = glob.glob('*VV_01*') #define all VV
VH_files = glob.glob('*VH_01*') #define all VH
zips = glob.glob('*.zip') # define the zip files

# loops to remove them

for f in HV_files:
    #os.remove(f)
    print(f)
    
for f in VV_files:
    os.remove(f)
    print(f)
    
for f in VH_files:
    os.remove(f)
    print(f)
    
for f in zips:
    os.remove(f)
    print(f)

lowman_05208_21015_009_210303_L090_CX_01_grd.zip


In [9]:
# check to see what files are left in the directory

print(glob.glob("/tmp/*.ann"))

['/tmp/lowman_05208_21015_009_210303_L090_CX_01.ann']


In [12]:
# First, let's print the annotation file to get a look at it's content
# these file contain a lot of information and can be very intimidating and hard to understand, but being able to read them is vital to working this UAVSAR data
for ann_file in glob.glob("/tmp/*.ann"):
    f = open(ann_file, 'r')
    file_contents = f.read()
    print (file_contents)
    #!head -n 15 /tmp/grmesa_27416_20003-028_20005-007_0011d_s01_L090HH_01.ann

; Parameter file for lowman_05208_21015_009_210303_L090_CX_01
;
; Search for parameters/value rather than placement in file
;
; slc = single look complex slant range image
; mlc = multi look cross product slant range image
; dat = compressed stokes matrix of multi-looked data
; grd = ground range projected (equiangular) and multi-looked data
; hgt = digital elevation model (DEM) used during processing and ground projection
; inc = ground range local incidence angle, the angle between the surface normal and the radar line of sight 
; slope = ground range terrain slope containing the derivatives of the DEM in the East and North direction, used to compute the local incidence angle
; h5  = HDF5 file containing binary data products mlc, grd, hgt, inc, and slope
; slc_mag and slc_phase are derived from the same 8 bytes per pixel of the slc input file
; mlc_mag and mlc_phase are derived from the same 8 bytes per pixel of the complex mlc input files
; grd_mag and grd_phase are ground range pro

In [19]:
from snowexsql.metadata import read_InSar_annotation
from snowexsql.string_management import *
import pytz

for ann_file in glob.glob("/tmp/*.ann"):
    with open(ann_file) as fp:
        lines = fp.readlines()
        fp.close()

    data = {}

    # loop through the data and parse
    for line in lines:

        # Filter out all comments and remove any line returns
        info = line.strip().split(';')
        comment = info[-1].strip().lower()
        info = info[0]

        # ignore empty strings
        if info and "=" in info:
            d = info.split('=')
            name, value = d[0], d[1]

            # Clean up tabs, spaces and line returns
            key = name.split('(')[0].strip().lower()
            units = get_encapsulated(name, '()')
            if not units:
                units = None
            else:
                units = units[0]

            value = value.strip()

            # Cast the values that can be to numbers ###
            if value.strip('-').replace('.', '').isnumeric():
                if '.' in value:
                    value = float(value)
                else:
                    value = int(value)

            # Assign each entry as a dictionary with value and units
            data[key] = {'value': value, 'units': units, 'comment': comment}

    # Convert times to datetimes
    for timing in ['start', 'stop']:
        key = '{} time of acquisition'.format(timing)
        dt = pd.to_datetime(data[key]['value'])
        dt = dt.astimezone(pytz.timezone('US/Mountain'))
        data[key]['value'] = dt

In [21]:
desc = data

In [41]:
import rasterio
from rasterio.crs import CRS
from rasterio.transform import Affine
import numpy as np
from os.path import basename, join, isdir
from snowexsql.utilities import get_logger

log = get_logger('grd2tif')
directory = '/tmp/'
output = '/tmp/grd_out/'
temp = join(output, 'temp')
clean_first = False

for d in [output, temp]:
    if isdir(d):
        if clean_first:
            log.info('Removing {}...'.format(d))
            shutil.rmtree(d)

    if not isdir(d):
        os.mkdir(d)

for ann in glob.glob("/tmp/*.ann"):
    # Form a pattern based on the annotation filename
    base_f = basename(ann)
    pattern = '.'.join(base_f.split('.')[0:-1])
    pattern = ('_'.join(pattern.split('_')[0:6]))+ '*.grd'
    # Gather all files associated
    grd_files = glob.glob(join(directory, pattern))
    grd_files = set(grd_files)

    log.info(
        'Converting {} grd files to geotiff...'.format(
            len(grd_files)))

    for grd in grd_files:

        # Save to our temporary folder and only change fname to have
        # ext=tif
        latlon_tiff = grd.replace(directory, temp).replace('grd', 'tif')
        grd_file = grd
        desc = desc 
        out_file =latlon_tiff

        log = get_logger('insar_2_raster')

        data_map = {'int': 'interferogram',
                    'amp1': 'amplitude of pass 1',
                    'amp2': 'amplitude of pass 2',
                    'cor': 'correlation',
                   'unw':'unwrapped','hgt':'heights'}

        # Grab just the filename and make a list splitting it on periods
        fparts = basename(grd_file).split('.')
        print(fparts)
        fkey = fparts[0]
        print(fkey)
        ftype = fparts[-2]
        print(ftype)
        dname = data_map[ftype]
        print(dname)
        if dname == 'unwrapped' or dname == 'heights':
            pass
        else:
            log.info('Processing {} file...'.format(dname))

            # Grab the metadata for building our georeference
            nrow = desc['ground range data latitude lines']['value']
            ncol = desc['ground range data longitude samples']['value']

            # Find starting latitude, longitude already at the center
            lat1 = desc['ground range data starting latitude']['value']
            lon1 = desc['ground range data starting longitude']['value']

            # Delta latitude and longitude
            dlat = desc['ground range data latitude spacing']['value']
            dlon = desc['ground range data longitude spacing']['value']
            log.debug('Expecting data to be shaped {} x {}'.format(nrow, ncol))

            log.info('Using Deltas for lat/long = {} / {} degrees'.format(dlat, dlon))

            # Read in the data as a tuple representing the real and imaginary
            # components
            log.info(
                'Reading {} and converting it from binary...'.format(
                    basename(grd_file)))

            bytes = desc['{} bytes per pixel'.format(dname.split(' ')[0])]['value']
            log.info('{} bytes per pixel = {}'.format(dname, bytes))

            # Form the datatypes
            if dname in 'interferogram':
                # Little Endian (<) + real values (float) +  4 bytes (32 bits) = <f4
                dtype = np.dtype([('real', '<f4'), ('imaginary', '<f4')])
            else:
                dtype = np.dtype([('real', '<f{}'.format(bytes))])

            # Read in the data according to the annotation file and bytes
            z = np.fromfile(grd_file, dtype=dtype)
            print(z)

            # Reshape it to match what the text file says the image is
            z = z.reshape(nrow, ncol)
            print(z)

            # Build the transform and CRS
            crs = CRS.from_user_input("EPSG:4326")

            # Lat1/lon1 are already the center so for geotiff were good to go.
            t = Affine.translation(lon1, lat1) * Affine.scale(dlon, dlat)
            ext = out_file.split('.')[-1]
            fbase = join(
                dirname(out_file), '.'.join(
                    basename(out_file).split('.')[
                        0:-1]) + '.{}.{}')

            for i, comp in enumerate(['real', 'imaginary']):
                print(i)
                print(comp)
                if comp in z.dtype.names:
                    d = z[comp]
                    out = fbase.format(comp, ext)
                    log.info('Writing to {}...'.format(out))
                    dataset = rasterio.open(
                        out,
                        'w+',
                        driver='GTiff',
                        height=d.shape[0],
                        width=d.shape[1],
                        count=1,
                        dtype=d.dtype,
                        crs=crs,
                        transform=t,
                    )
                    # Write out the data
                    dataset.write(d, 1)

                    # show(new_dataset.read(1), vmax=0.1, vmin=-0.1)
                    # for stat in ['min','max','mean','std']:
                    #     log.info('{} {} = {}'.format(comp, stat, getattr(d, stat)()))
                    dataset.close()



grd2tif INFO Converting 6 grd files to geotiff...


['lowman_05208_21015_009_210303_L090HVVV_CX_01', 'grd']
lowman_05208_21015_009_210303_L090HVVV_CX_01
lowman_05208_21015_009_210303_L090HVVV_CX_01


KeyError: 'lowman_05208_21015_009_210303_L090HVVV_CX_01'

In [28]:
join(directory, pattern)

'/tmp/lowman_05208_21015_009_210303_L090_CX_01*.grd'